In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
import numpy as np 
import pandas as pd 
import sys 
from scipy.interpolate import interp1d
from scipy.interpolate import splrep as splrep

sys.path.append('/home/ak/Documents/Research/master/hsmm_core/')
sys.path.append('/home/ak/Documents/Research/master/')
from hsmm_core.hmm import * 

from hsmm_core.prediction_engines import *  
import hsmm_core.observation_models 
from test_hmm.test_utils import generate_sample_paths_independent_sets
from hsmm_core.observation_models import ExpIndMixDiracGauss
%matplotlib inline

Let us create a fairly distinguishable situation : Two states with skewed transition matrix favouring one state 
and static parameters that are sufficiently apart

In [3]:
n_hidden_states = 2
sigmas = [0.025, 0.45] 
lambdas = [0.5, 2] 
weights = [0.4, 0.3] 

tpm = np.array([[0.6, 0.4], [0.3, 0.7]])
startprob = np.array([0.8, 0.2]) 
obs_model = hsmm_core.observation_models.ExpIndMixDiracGauss(n_hidden_states)
obs_model.sigmas_ = sigmas  
obs_model.lambdas_ = lambdas  
obs_model.weight_ = weights 

the_hmm = hmm_engine(obs_model, n_hidden_states) 

the_hmm.pi_ = startprob
the_hmm.tpm_ = tpm

In [4]:
print "sigmas ", obs_model.sigmas_
print "lambdas ", obs_model.lambdas_

print "Transition matrix ", the_hmm.tpm_

sigmas  [0.025, 0.45]
lambdas  [0.5, 2]
Transition matrix  [[0.6 0.4]
 [0.3 0.7]]


# Helper Funcions for Plotting


In [5]:
def custom_lineplot(ax, x, y, error, xlims, ylims, color='red'):
    """Customized line plot with error bars."""
    
    ax.errorbar(x, y, yerr=error, color=color, ls='--', marker='o', capsize=5, capthick=1, ecolor='black')
    
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    
    return ax
    
def custom_scatterplot(ax, x, y, error, xlims, ylims, color='green', markerscale=100):
    """Customized scatter plot where marker size is proportional to error measure."""
    
    markersize = error * markerscale
    
    ax.scatter(x, y, color=color, marker='o', s=markersize, alpha=0.5)
    
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    
    return ax
    
def custom_barchart(ax, x, y, error, xlims, ylims, error_kw, color='lightblue', width=0.75):
    """Customized bar chart with positive error bars only."""
    
    error = [np.zeros(len(error)), error]
    
    ax.bar(x, y, color=color, width=width, yerr=error, error_kw=error_kw, align='center')
    
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    
    return ax
    
def custom_boxplot(ax, x, y, error, xlims, ylims, mediancolor='magenta'):
    """Customized boxplot with solid black lines for box, whiskers, caps, and outliers."""
    
    medianprops = {'color': mediancolor, 'linewidth': 2}
    boxprops = {'color': 'black', 'linestyle': '-'}
    whiskerprops = {'color': 'black', 'linestyle': '-'}
    capprops = {'color': 'black', 'linestyle': '-'}
    flierprops = {'color': 'black', 'marker': 'x'}
    
    ax.boxplot(y,
               positions=x,
               medianprops=medianprops,
               boxprops=boxprops,
               whiskerprops=whiskerprops,
               capprops=capprops,
               flierprops=flierprops)
    
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    
    return ax

In [6]:
def states_from_fixed_ratios(ratios, total_length):
    states = np.array([], dtype=np.int64) 
    ratios_ids = np.arange(len(ratios))
    rng = np.random.RandomState(345)
    while len(states) < total_length: 
        
        ratio = ratios[rng.choice(ratios_ids)] 
        #print ratio
        states = np.append(states, np.append(np.repeat(0, 100*ratio[0]), np.repeat(1, 100*ratio[1]))) 
        #print len(states)
    return states

# Lets fix a ratio of 20:80 and sample some fixed states 
state_ratios = np.array([[0.2, 0.05], [0.4, 0.1], [0.8, 0.2]]) 

# Lets fix a ratio of 20:80 and sample some fixed states 
state_ratios = np.array([[0.2, 0.05], [0.4, 0.1], [0.8, 0.2]]) 

Ts = [ 5, 10, 20, 50, 70, 100, 200,300, 400,500, 700, 900,1100, 1200,1300,1400, 1500]
M = 10000

all_errors = np.zeros((len(Ts), M))

In [7]:
###create sequences###
for i_T, T in enumerate(Ts):
    rng = np.random.RandomState(12345)
    states = states_from_fixed_ratios(state_ratios, T)
    
    for m in xrange(M):
        random_states = the_hmm.sample_states(rng=rng, length=M)
        observation_points = obs_model.sample_data(M, rng=rng, state=states)
        viterbi_inferred = the_hmm.map_metrics(observation_points)['viterbi_optimal_state_seq'] 
        all_errors[i_T, m] = np.sum((viterbi_inferred - states)**2)/float(T) 
        # The first duration is always zero
        observation_points[0, 0] = 0.

ValueError: Not all neccessary params are provided

We create state dequences of fixed length 100, 400, 700, 900, 1200, 1500 and for every fixed length generate 
10e4 copies of observatio data from the above distribution. 

In [11]:
###create sequences###
for i_T, T in enumerate(Ts):
    rng = np.random.RandomState(12345)
    states = states_from_fixed_ratios(state_ratios, T)
    # uncomment here to generate states from the hidden process distribution directly. 
    states, _ = the_hmm.sample_states(rng=rng, length=T)
    for m in xrange(M):
        obs_samples = obs_model.sample_data(M, states, rng=rng)
        
        viterbi_inferred = the_hmm.map_metrics(obs_samples)['viterbi_optimal_state_seq'] 
        all_errors[i_T, m] = np.sum((viterbi_inferred - states)**2)/float(T)  
        print ("sequence no:",i_T,"length of sequence:",T,"copies of sequence:",M)
        print'###ERROR###'
        print all_errors[i_T,m]
        print '####'

ValueError: need more than 1 value to unpack

In [8]:
tck, u = splrep(pts.T, u=None, s=0.0, per=1) 
u_new = np.linspace(u.min(), u.max(), 1000) 
x_new, y_new = splev(u_new, tck, der=0)

plt.plot(pts[:,0], pts[:,1], 'ro') 
plt.plot(x_new, y_new, 'b--') 
plt.show()

NameError: name 'pts' is not defined

In [8]:
Ts =np.asarray(Ts)
Ts.shape
# define pts from the question

tck, u = splprep(pts.T, u=None, s=0.0, per=1) 
u_new = np.linspace(u.min(), u.max(), 1000)
x_new, y_new = splev(u_new, tck, der=0)

plt.plot(pts[:,0], pts[:,1], 'ro')
plt.plot(x_new, y_new, 'b--')
plt.show()



NameError: name 'final_errors' is not defined

In [9]:
def func_determinant(features, i):
    for i in range(0,3):
        _matrix=np.dot(features[i].T,features[i])
        _determinant = np.linalg.det(_matrix)
    return _determinant
    
def symmetric_matrix(features, i):
    for i in range(0,3):
        _sym_mat[i]= np.dot(features[i].T,features[i])

In [10]:
def squareform_diagfill(arr1D): 
    n = int(np.sqrt(arr1D.size*2)) 
    if (n*(n+1))//2!=arr1D.size: 
        print "Size of 1D array not suitable for creating a symmetric 2D array!" 
        return None 
    else: 
        R,C = np.triu_indices(n) 
        out = np.zeros((n,n),dtype=arr1D.dtype) 
        out[R,C] = arr1D 
        out[C,R] = arr1D 
        return out

In [9]:
state_ratios = np.array([[0.2, 0.05], [0.4, 0.1], [0.8, 0.2]]) 

Ts = [100, 200, 300, 400, 500,600, 700, 1000,2000]
M = 10000 

#all_determinants = np.zeros((len(Ts), M))
f_eng= hmm_features(hmm=the_hmm)
_inf_matrix={}
_ksi ={}


for i_T, T in enumerate(Ts):
    rng = np.random.RandomState(12345) #inside to make the sequences nested
    states = states_from_fixed_ratios(state_ratios, T)
    print len(states), i_T, T

100 0 100
250 1 200
300 2 300
475 3 400
525 4 500
625 5 600
725 6 700
1000 7 1000
2050 8 2000


In [12]:
from collections import OrderedDict 

state_ratios = np.array([[0.2, 0.05], [0.4, 0.1], [0.8, 0.2]]) 

T = 200
M = 100

#all_determinants = np.zeros((len(Ts), M))
f_eng= hmm_features(hmm=the_hmm)
_inf_matrix=OrderedDict()
_ksi =OrderedDict()

rng = np.random.RandomState(12345) #inside to make the sequences nested
states = states_from_fixed_ratios(state_ratios, T)
for m in xrange(M):
    _obs_samples = obs_model.sample_data(states, rng=rng)
    _df = pd.DataFrame(_obs_samples, columns=['Duration', 'ReturnTradedPrice'])
    _features = f_eng.generate_features(_df)
    _inf_matrix[m]=_features[1] #information matrix
    _ksi[m] =_features[3] #ksi

ValueError: Shape of passed values is (3, 250), indices imply (2, 250)

In [13]:
inf_keys =_inf_matrix.keys()
ln_T =T
#len(_inf_matrix)
spectral_gap = OrderedDict()
_trace= OrderedDict()
_determinant =OrderedDict()
for key in inf_keys:
    for i_t in range(0,ln_T):
        _s=np.linalg.svd(squareform_diagfill(np.asarray(_inf_matrix[key][i_t:i_t+1])), full_matrices= False, compute_uv= False)
        spectral_gap[(key, i_t)]=_s.max() - _s.min()
        _trace[(key, i_t)] =np.log(np.sum(_s)/max(1,key))
        _determinant[(key, i_t)] = np.log(np.prod(_s)/max(1,key))

In [14]:
list_keys =_determinant.keys()

In [15]:
len(list_keys)/200

0

In [18]:
_folder ='home/ak/Documents/Data'

In [22]:
sys.path.append(_folder)